In [2]:
from opensky_api import OpenSkyApi
import folium

# dont screw me here pls
# Replace with your OpenSky credentials
api = OpenSkyApi('JetRider', 'wiwnib-bizsez-5daTjo') 



# Provided tail numbers and their corresponding Mode S/ICAO24 codes
# TAIL_NUMBER_TO_ICAO24 = {
#     "N874SA": "ac06C3",
#     "N853SA": "abb40e",
#     "N828SA": "ab5024",
#     "N821SA": "ab3623"
# }


# List of icao24 codes for the aircrafts you are interested in
icao24_list = ['ac06C3']

# Get the states of the aircrafts in the icao24 list
states = api.get_states(icao24=icao24_list)

# Get the flight data by aircraft for the time interval mentioned
flights_by_aircraft = api.get_flights_by_aircraft(icao24_list, 1517227200, 1517230800)
print(flights_by_aircraft)

# Check if we have any states
if states and states.states:
    for s in states.states:
        # Set the current state
        current_state = s
        print('Current Position')
        print(current_state)

    # Get the track of a specific aircraft (I noticed you used a different icao24 here, adjust as necessary)
    track = api.get_track_by_aircraft("a0fe7a")
    print('Track')
    print(track)

    # Get the path data from the track
    path_data = track.path

    # Extract latitudes and longitudes from the path data
    latitudes = [point[1] for point in path_data]
    longitudes = [point[2] for point in path_data]

    # Calculate the average latitude and longitude to center the map
    avg_latitude = sum(latitudes) / len(latitudes)
    avg_longitude = sum(longitudes) / len(longitudes)

    # Create a map centered around the average latitude and longitude
    m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=8)

    # Plot the flight path on the map
    flight_path = list(zip(latitudes, longitudes))
    folium.PolyLine(flight_path, color="blue", weight=2.5, opacity=1).add_to(m)

    # Add a marker for the current position of the aircraft
    current_position = {
        'latitude': current_state.latitude,
        'longitude': current_state.longitude
    }
    folium.Marker([current_position['latitude'], current_position['longitude']], tooltip="Current Position").add_to(m)

    # Display the map
    m
else:
    print("No states found for the given icao24 list.")



None
No states found for the given icao24 list.
